# Modellierung Windeinspeisung

**Lernziele**
- Hyperparameteroptimierung mit keras-tuner

**!!!!Hinweis!!!!
Sie müssen bei den Paketen den Pfad zu den "internalfunctions" richtig setzen

## Pakete und Daten laden

In [ ]:
# Datenorganisation
import pandas as pd
import numpy as np
import datetime as dt

# Ploterstellung
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns

# Datenvorbereitung
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split

# Standardeinstellungen
plt.rcParams['axes.xmargin'] = 0
pd.set_option('display.precision',3)
np.set_printoptions(precision=3)

# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, LSTM, Dense, Bidirectional, Dropout)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint)
from tensorflow import keras


#tf.config.list_physical_devices('GPU')
#tf.debugging.set_log_device_placement(True)
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())
import keras_tuner as kt

import tensorboard
tensorboard.__version__

#sonst  Module 
import copy
import sys
from math import ceil

# Eingene Module 
# temporäre Einbinden des neuen Pfades mit zu ladenden Dateien
sys.path.append("C:\\Users\\rs3753e\\sciebo\\Vorlesungen\\SoSe_Energiedatenanalyse Datamining\\internalfunctions\\")
from KNN import KNN

In [ ]:
# WICHTIG: alte Verzeichnis-Daten werden gelöscht

# Laden des vorbereiteten WinddatenFrame
wind=pd.read_hdf('../daten/windeinspeisung_bereinigt','Obj1')
wind

## Datenorganisation

Erstellung eines KNN-Objektes mit normierten/Standardisierten Trainings/Validierungs- und Testdaten

In [27]:
# Vorgaben zur Datentrennung
sections=(.6,.25,.15)
shuffle=False

# Vorbereitete Klasse
model = KNN()
# Unterteilung der Samples
model.split_sample(pd.DataFrame(wind["Ws_avg"]),pd.DataFrame(wind["S_avg"]),shuffle=shuffle,sections =sections)
#Auswahl der Skalierungsmethode
model.scaler("MinMax")

## Erstellung eines KNN zur Prognose der Einspeisung

**Definition Features und Output**

In [28]:
model.numFeatures = 1
model.numResponses = 1;

In [29]:
model.x[0].shape

(121071, 1)

### Modellaufbau mit Keras-functional API Schreibweise

***HyperParameters (hp)***

Die Suche der Hyperparamters muss mit der Tuner API eingenständig definiert werden.<br>

Hierzu stehen 4 Variationsmöglichkeiten bei den (Hyper)-Parametern zur Verfügung:

*   `Boolean` —  Choice between True and False.<br>
     hp.Boolean(name, default=False,...)    

*   `Choice` —  Choice of one value among a predefined set of possible values.<br>
     hp.Choice(
    name, values, ordered=None, default=None,...)

*   `Int` —  Integer point value hyperparameter.<br>
    hp.Int('units', min_value=32, max_value=128, step=32,default=64)

*   `Float` —  Floating point value hyperparameter.<br>
     hp.Float(name,min_value,max_value,step=None,sampling="linear",default=None,...)


*   `conditional_scope` —  Opens a scope to create conditional HyperParameters.<br> 

Im  Beispiel werden folgende Parameter variiert (Variationsmehtode in Klammern):
- units (Int)
- Anzahl der hidden-Layer (Int)
- Learning rate (Choice)
- Layerdropout (Boolean) 

In [30]:
# Methode zur Auswahl der zu variierenden  Hyperparameter
def build_model(hp):                                                                #     hp ist Objekt von Keras Tuner zur Hyperparameteroptimierung
    
    # Erstellung der relevanten Parametern und deren Variationen
    
    units = hp.Int("units", min_value=32, max_value=512, step=32)                   # Anzahl Neuronen
    layers = hp.Int("layers", min_value=1, max_value=2, step=1)                     # Anzahl der Layer
    activation = hp.Choice("activation", ["relu", "tanh"])                          # Form der Aktivierung
    dropout = hp.Boolean("dropout")                                                 # dropout ja / nein
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")             # Lernrate
    
    
    # Aufruf der Modellerstellung auf Basis der gewählten Hyperparameter 
    KNN = single_call(
        units=units,layers=layers, activation=activation, dropout=dropout, lr=lr
    )
    # Rückgabe des Netzes
    return KNN

***dynamischer Aufbau der Netztopologie***

Für die jeweiligen Parameterkonstellationen, welche bei der (Hyper)-Parametersuche in Frage kommen, wird ein eigenständiger Methodenaufruf definiert mit Übergabeobjekt `hp`. 

Für die Erstellung des neuronalen Netzes wird die funktionale API verwendet.

In [31]:
# Modellaufbau mit gewählter Hyperparameterausstattung
def single_call(units, layers,activation, dropout, lr):
     
    # Define model layers.
    input_layer = Input(shape=(model.numFeatures,))
    
    X= Dense(units=units,activation=activation,name='dense1')(input_layer)
    if layers >1:
        X = [Dense(units=units,activation=activation,\
                             name='dense'+str(i+2))(X) for i in range(layers-1)]
    if dropout==True:
        X = Dropout(rate=0.25)(X)
    
    # Output
    y_output= Dense(units='1', name='output')(X)  
    
    # Verbindung von Input und Output-Schicht   
    KNN = Model(inputs=input_layer,outputs=y_output)

    KNN.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                loss=keras.losses.MeanSquaredError(),
                metrics=keras.losses.MeanAbsoluteError())
    
    return KNN

**Initialisierung des Tuners und Durchführung des hypertunings**

The base Tuner class is the class that manages the hyperparameter search process, including model creation, training, and evaluation. For each trial, a Tuner receives new hyperparameter values from an Oracle instance. After calling model.fit(...), it sends the evaluation results back to the Oracle instance and it retrieves the next set of hyperparameters to try.

Es stehen in Keras 3 verschiedene Möglichkeiten der Hyperparameteroptimierung zur Verfügung. die Verfahren unterscheiden sich im Wesentlichen durch die Form des Suchalgorithmus (siehe Vorlesungsunterlagen)

- RandomSearch            
- BayesianOptimization
- Hyperband

**Argumente der Initialisierung**

Bei Instanzierung einer definierten Suchalgorithmus-Klasse können entsprechende Argumente übergeben werden 
```python
keras_tuner.BayesianOptimization(
    hypermodel=None,
    objective=None,
    max_trials=10,
    num_initial_points=None,
    alpha=0.0001,
    beta=2.6,
    seed=None,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True,
    max_retries_per_trial=0,
    max_consecutive_failed_trials=3,
    **kwargs
)

``` 
Wir übergeben hierbei die Argumente `hypermodel` und `input_shape` zu übergeben.
*   `hypermodel` —  übergibt den Modellaufbau (in unseren Fall die Methode build_model)

*   `objective` — verwendete Zielfunktion in der Suche. Wichtig ist es hierbei eine Verlustfunktion auf Basis der Validierungsdaten zu verwenden

*   `max_trials:` — Anzahl der Versuche (model configurations) 


**Umsetzung der Initialisierung**

In [32]:
kwarg = {'project_name':'basian_search','overwrite': True}

tuner = kt.BayesianOptimization(build_model,
                     objective='val_loss',
                     max_trials=40,
                     **kwarg)

# Zusammenfassung
tuner.search_space_summary()

Search space summary
Default search space size: 5
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


**bereitgestellte Methoden des tuner-Objektes** 

*    `search`(*fit_args, **fit_kwargs) - Performs a search for best hyperparameter configuations.

*    `results_summary` - Print search space summary.

*    `search_space_summary`(*fit_args, **fit_kwargs) - Performs a search for best hyperparameter configuations.

*    `get_best_hyperparameters` - Returns the best hyperparameters, as determined by the objective.<br> **Arguments:**  num_trials: Optional number of HyperParameters objects to return. <br>**Returns:** List of HyperParameter objects sorted from the best to the worst.

*    `get_best_models` - Returns the best model(s), as determined by the tuner's objective.

*    `results_summary` - The method prints a summary of the search results including the hyperparameter values and evaluation results for each trial.
**Arguments:**  num_trials: Optional number of trials to display. Defaults to 10.

*    `load_model` - Loads a Model from a given trial.

**Durchführung der Parametersuche**

In [33]:
stop_early = EarlyStopping(monitor='val_loss', patience=10)
tuner.search(model.x_norm[0],model.y_norm[0],batch_size = 512,epochs =10,validation_data=(model.x_norm[1],model.y_norm[1]), callbacks=[stop_early])

Trial 40 Complete [00h 00m 09s]
val_loss: 0.005034321919083595

Best val_loss So Far: 0.0048936656676232815
Total elapsed time: 00h 06m 02s
INFO:tensorflow:Oracle triggered exit


**Sichtung der Ergebnisse**

In [34]:
#tuner.results_summary();

In [35]:
# Alternative A: Get the top model.
best_models = tuner.get_best_models()[0]

In [36]:
# Alternative B: Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

**Re-Trainieren des besten neuronalen Netzes**

In [37]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model_best = tuner.hypermodel.build(best_hps)
model_best.summary()
history = model_best.fit(model.x_norm[0],model.y_norm[0], epochs=100, batch_size=1000,validation_data=(model.x_norm[1],model.y_norm[1]),callbacks=[stop_early])

val_loss_per_epoch = history.history['val_loss']
best_epoch = val_loss_per_epoch.index(max(val_loss_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense1 (Dense)               (None, 384)               768       
_________________________________________________________________
output (Dense)               (None, 1)                 385       
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
122/122 [==============================] - 1s 5ms/step - loss: 0.0076 - mean_absolute_error: 0.0457 - val_loss: 0.0057 - val_mean_absolute_error: 0.0296
Epoch 2/100
122/122 [==============================] - 0s 3ms/step - loss: 0.0026 - mean_absolute_error: 0.0229 - val_loss: 0.0054 - val_mean_absolute_error: 0.0286
Epoch 3/100
122/122 [==============================] - 0s

**Plot Verlauf der Verlustfunktion**

In [38]:
fig, ax =plt.subplots(nrows=1,ncols=1,**{'figsize': (18, 7)})
# ersten 5 werden ausgeblendet
lin1 = ax.plot(history.history['val_loss'][5:],label='val_loss')
lin2 = ax.plot(history.history['loss'][5:],label='loss')
ax.set(xlabel='Epoche',ylabel='loss')
ax.legend();ax.grid()

**Prognose von Training, Validierungs- und Testdaten**

In [39]:
# Prognose
ypred_norm =list();[ypred_norm.append(model_best.predict(model.x_norm[i])) for i in range(3)];
# Inverse Scalierung
ypred =list();[ypred.append(model.scalerY.inverse_transform(ypred_norm[i])) for i in range(3)];

**Verlauf Plot Orginal vs. Prognose**

In [40]:
lab =["Training", "Validierung","Test"]
fig, ax =plt.subplots(nrows=1,ncols=3,**{'figsize': (18, 8)})
for i in range(3):
  
    # 50% Quantil'
    ax[i].plot(model.x[i].index[0:1000],ypred[i][0:1000],label='pred',color = 'r',linewidth =.5)

    # realer Wert
    ax[i].plot(model.x[i].index[0:1000],model.y[i][0:1000],label='real',color = 'k',linewidth =.5)
    ax[i].set(ylabel="Leistung [MW]",xlabel="Zeit",title=lab[i],ylim=(0,2200));
    ax[i].legend();